In [91]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [92]:
import torch
import numpy as np
import src.models
from exp_ntk import run

In [110]:
def get_str_l(l, precision=2):
    l_str = []
    for el in l:
        l_str.append(f"{el:.{precision}f}")
    return l_str
    

In [105]:
seed = 0

data_params = dict(
    learning_setting = "inductive", # or "transdructive"
    classes = 2,
    n_trn_labeled = 600,
    n_trn_unlabeled = 0,
    n_val = 200,
    n_test = 200,
    sigma = 1,
    avg_within_class_degree = 1.58 * 2,
    avg_between_class_degree = 0.37 * 2,
    K = 1.5,
    seed = 0 # used to generate the dataset & data split
)

model_params = dict(
    label = "GCN",
    model = "GCN",
    normalization = "row_normalization",
    depth = 1,
)

attack_params = dict(
    attack = "random",
    #epsilon_list = [0, 0.01, 0.025, 0.05, 0.10, 0.25, 0.50, 1, 2.5, 5, 10],
    epsilon_list = [0],
    attack_setting = "evasion" # or "poisioning"
)

verbosity_params = dict(
    debug_lvl = "warning"
)  

other_params = dict(
    device = "0",
    dtype = torch.float64,
    allow_tf32 = False
)

In [106]:
data_params["learning_setting"] = "transductive"
data_params["seed"] = 0
run(data_params, model_params, attack_params, verbosity_params, other_params, seed)

0.006326326326326326
0.0014814814814814814


{'accuracy_l': [0.6949999928474426],
 'epsilon_l': [0],
 'min_ypred': [-5.432753155544628],
 'max_ypred': [9.904927987945971],
 'min_M': [-1.7680838899133418],
 'max_M': [2.993227987220207],
 'min_ntklabeled': [-1.939445211826925],
 'max_ntklabeled': [63.12027260267322],
 'min_ntkunlabeled': [-2.2044594382782496],
 'max_ntkunlabeled': [26.982307255502207],
 'cond': 751303.9586359197}

## Experiments for NTK matrices sizes

In [111]:
get_str_l(acc_l)

['0.80',
 '0.79',
 '0.83',
 '0.79',
 '0.74',
 '0.82',
 '0.78',
 '0.75',
 '0.77',
 '0.78']

In [112]:
data_params["learning_setting"] = "transductive"
data_params["n_trn_labeled"] = 400
acc_l = []
min_ypred = []
max_ypred = []
cond = []
min_M = []
max_M = []
min_ntklabeled = []
max_ntklabeled = []
min_ntkunlabeled = []
max_ntkunlabeled = []
for seed in range(10):
    data_params["seed"] = seed
    res = run(data_params, model_params, attack_params, verbosity_params, other_params, seed)
    acc_l.append(res["accuracy_l"][0])
    min_ypred.append(res["min_ypred"][0])
    max_ypred.append(res["max_ypred"][0])
    min_M.append(res["min_M"][0])
    max_M.append(res["max_M"][0])
    min_ntklabeled.append(res["min_ntklabeled"][0])
    max_ntklabeled.append(res["max_ntklabeled"][0])
    min_ntkunlabeled.append(res["min_ntkunlabeled"][0])
    max_ntkunlabeled.append(res["max_ntkunlabeled"][0])
    cond.append(res["cond"])
print(f"Accuracy: {get_str_l(acc_l)}")
print(f"Min y_pred: {get_str_l(min_ypred)}")
print(f"Max y_pred: {get_str_l(max_ypred)}")
print(f"Min M: {get_str_l(min_M)}")
print(f"Max M: {get_str_l(max_M)}")
print(f"Min NTK_labeled: {get_str_l(min_ntklabeled)}")
print(f"Max NTK_labeled: {get_str_l(max_ntklabeled)}")
print(f"Min NTK_unlabeled: {get_str_l(min_ntkunlabeled)}")
print(f"Max NTK_unlabeled: {get_str_l(max_ntkunlabeled)}")
print(f"Condition: {get_str_l(cond, precision=0)}")

0.007909887359199
0.0018523153942428035
0.007909887359199
0.0018523153942428035
0.007909887359199
0.0018523153942428035
0.007909887359199
0.0018523153942428035
0.007909887359199
0.0018523153942428035
0.007909887359199
0.0018523153942428035
0.007909887359199
0.0018523153942428035
0.007909887359199
0.0018523153942428035
0.007909887359199
0.0018523153942428035
0.007909887359199
0.0018523153942428035
Accuracy: ['0.80', '0.79', '0.83', '0.79', '0.74', '0.82', '0.78', '0.75', '0.77', '0.78']
Min y_pred: ['-3.09', '-6.62', '-5.19', '-4.70', '-5.72', '-9.12', '-4.91', '-8.62', '-5.72', '-6.30']
Max y_pred: ['4.57', '7.56', '4.66', '4.09', '5.15', '4.81', '2.68', '3.77', '4.67', '4.54']
Min M: ['-1.28', '-2.77', '-1.51', '-35.60', '-1.49', '-1.49', '-1.71', '-1.64', '-130.30', '-1.48']
Max M: ['3.20', '3.05', '3.14', '35.61', '3.71', '3.05', '2.41', '2.74', '130.27', '3.24']
Min NTK_labeled: ['-0.74', '-1.50', '-0.93', '-1.32', '-1.48', '-1.29', '-0.93', '-1.34', '-1.81', '-1.84']
Max NTK_label

In [113]:
data_params["learning_setting"] = "inductive"
data_params["n_trn_labeled"] = 400
acc_l = []
min_ypred = []
max_ypred = []
cond = []
min_M = []
max_M = []
min_ntklabeled = []
max_ntklabeled = []
min_ntkunlabeled = []
max_ntkunlabeled = []
for seed in range(10):
    data_params["seed"] = seed
    res = run(data_params, model_params, attack_params, verbosity_params, other_params, seed)
    acc_l.append(res["accuracy_l"][0])
    min_ypred.append(res["min_ypred"][0])
    max_ypred.append(res["max_ypred"][0])
    min_M.append(res["min_M"][0])
    max_M.append(res["max_M"][0])
    min_ntklabeled.append(res["min_ntklabeled"][0])
    max_ntklabeled.append(res["max_ntklabeled"][0])
    min_ntkunlabeled.append(res["min_ntkunlabeled"][0])
    max_ntkunlabeled.append(res["max_ntkunlabeled"][0])
    cond.append(res["cond"])
print(f"Accuracy: {get_str_l(acc_l)}")
print(f"Min y_pred: {get_str_l(min_ypred)}")
print(f"Max y_pred: {get_str_l(max_ypred)}")
print(f"Min M: {get_str_l(min_M)}")
print(f"Max M: {get_str_l(max_M)}")
print(f"Min NTK_labeled: {get_str_l(min_ntklabeled)}")
print(f"Max NTK_labeled: {get_str_l(max_ntklabeled)}")
print(f"Min NTK_unlabeled: {get_str_l(min_ntkunlabeled)}")
print(f"Max NTK_unlabeled: {get_str_l(max_ntkunlabeled)}")
print(f"Condition: {get_str_l(cond, precision=0)}")

0.007909887359199
0.0018523153942428035
0.007909887359199
0.0018523153942428035
0.007909887359199
0.0018523153942428035
0.007909887359199
0.0018523153942428035
0.007909887359199
0.0018523153942428035
0.007909887359199
0.0018523153942428035
0.007909887359199
0.0018523153942428035
0.007909887359199
0.0018523153942428035
0.007909887359199
0.0018523153942428035
0.007909887359199
0.0018523153942428035
Accuracy: ['0.54', '0.33', '0.76', '0.38', '0.49', '0.56', '0.57', '0.32', '0.68', '0.56']
Min y_pred: ['-97490.87', '-48360.84', '-50460.89', '-28497.31', '-142206.29', '-19802.02', '-28167.63', '-40436.56', '-75658.87', '-40327.67']
Max y_pred: ['182670.60', '54618.05', '55201.70', '10045.66', '115411.53', '7496.70', '32783.82', '47047.44', '23566.33', '10303.28']
Min M: ['-35904.57', '-20096.42', '-19311.67', '-23325.17', '-64763.02', '-13258.93', '-42183.66', '-20909.61', '-108140.02', '-161703.38']
Max M: ['61231.26', '20089.39', '24244.20', '23340.64', '64742.14', '20135.12', '42191.24',

## Experiments for y_pred size

In [71]:
data_params["learning_setting"] = "transductive"
data_params["n_trn_labeled"] = 400
acc_l = []
min_ypred = []
max_ypred = []
cond = []
for seed in range(10):
    data_params["seed"] = seed
    res = run(data_params, model_params, attack_params, verbosity_params, other_params, seed)
    acc_l.append(res["accuracy_l"][0])
    min_ypred.append(res["min_ypred"][0])
    max_ypred.append(res["max_ypred"][0])
    cond.append(res["cond"])
print(acc_l)
print(min_ypred)
print(max_ypred)
print(cond)

[0.7999999523162842, 0.7949999570846558, 0.8299999833106995, 0.7949999570846558, 0.7400000095367432, 0.824999988079071, 0.7799999713897705, 0.75, 0.7749999761581421, 0.7799999713897705]
[-3.0920467578784803, -6.6160343128658, -5.194746775288973, -4.699020576493933, -5.718078334268294, -9.115392589081491, -4.905361471048922, -8.61613686394207, -5.719317177977954, -6.302207013707667]
[4.570836528482572, 7.558606664491745, 4.6590389998099635, 4.0878273364676465, 5.15241891023212, 4.80595657486644, 2.6751935812229077, 3.7679468135510303, 4.6712427287199105, 4.540205492559701]
[389053.63117462175, 17227532.286783576, 386959.6250253748, 175422543.52912518, 382813.985553917, 397663.6694353313, 367239.72382285836, 412507.3546088346, 678918974.0532519, 413583.01504568243]


In [102]:
data_params["learning_setting"] = "inductive"
p_unnorm = data_params["avg_within_class_degree"]
data_params["avg_within_class_degree"] = 1.58 * 2 * 100
q_unnorm = data_params["avg_between_class_degree"]
data_params["avg_between_class_degree"] = 0.37 * 2 * 70
acc_l = []
min_ypred = []
max_ypred = []
cond = []
for seed in range(10):
    data_params["seed"] = seed
    res = run(data_params, model_params, attack_params, verbosity_params, other_params, seed)
    acc_l.append(res["accuracy_l"][0])
    min_ypred.append(res["min_ypred"][0])
    max_ypred.append(res["max_ypred"][0])
    cond.append(res["cond"])
print(acc_l)
print(min_ypred)
print(max_ypred)
print(cond)

0.6326326326326326
0.1037037037037037
0.6326326326326326
0.1037037037037037
0.6326326326326326
0.1037037037037037
0.6326326326326326
0.1037037037037037
0.6326326326326326
0.1037037037037037
0.6326326326326326
0.1037037037037037
0.6326326326326326
0.1037037037037037
0.6326326326326326
0.1037037037037037
0.6326326326326326
0.1037037037037037
0.6326326326326326
0.1037037037037037
[1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0]
[-198887.16105522725, -570682.9825172557, -1447257.8573979978, -37860.66165276896, -55926.801587639144, -290040.57506164117, -350577.9186986564, -391894.0606868819, -590375.6752174892, -351662.7199801542]
[320796.28391054017, 394927.00222813245, 1027750.7295307582, 28479.814787887735, 138296.76934363693, 467328.4684214182, 303189.61444266746, 288720.0622061163, 543805.0602562819, 1210417.7903377097]
[218709333701.72552, 395523266595.4415, 4047205108377.5347, 367468484718.35284, 328162745884.328, 297005777329.7517, 252244108489.32474, 298833724882.89355, 111525099

In [ ]:
data_params["learning_setting"] = "transductive"
acc_l = []
min_ypred = []
max_ypred = []
cond = []
for seed in range(10):
    data_params["seed"] = seed
    res = run(data_params, model_params, attack_params, verbosity_params, other_params, seed)
    acc_l.append(res["accuracy_l"][0])
    min_ypred.append(res["min_ypred"][0])
    max_ypred.append(res["max_ypred"][0])
    cond.append(res["cond"])
print(acc_l)
print(min_ypred)
print(max_ypred)
print(cond)

-> clear correlation low y_pred values mean good predictions (|min/max| < 10)

In [68]:
data_params["learning_setting"] = "inductive"
acc_l = []
min_ypred = []
max_ypred = []
cond = []
for seed in range(10):
    data_params["seed"] = seed
    res = run(data_params, model_params, attack_params, verbosity_params, other_params, seed)
    acc_l.append(res["accuracy_l"][0])
    min_ypred.append(res["min_ypred"][0])
    max_ypred.append(res["max_ypred"][0])
    cond.append(res["cond"])
print(acc_l)
print(min_ypred)
print(max_ypred)
print(cond)

[0.6899999976158142, 0.5550000071525574, 0.5799999833106995, 0.4449999928474426, 0.3349999785423279, 0.675000011920929, 0.44999998807907104, 0.44999998807907104, 0.6100000143051147, 0.5]
[-30247.846224752207, -95078.36257414054, -26783.68457652904, -21873.26632158264, -48363.07600911075, -28243.6273096534, -16259.459278168943, -33341.27322340436, -24276.7436774647, -45002.83794352863]
[120458.61990854447, 46644.489896458486, 96389.28003963197, 15950.742201795094, 16671.90547960927, 16726.042436049684, 56953.97201926261, 32503.00367441383, 19232.59269619555, 50973.9443629398]
[481320232.12236416, 904617224.7448723, 390462481.61452425, 108613454.24243383, 443483057.5212938, 171410416.884777, 640581624.5041744, 218038721.29718912, 216075456.717386, 216347577.68922815]


In [75]:
data_params["learning_setting"] = "transductive"
data_params["n_trn_labeled"] = 2000
data_params["n_val"] = 500
data_params["n_test"] = 1000
acc_l = []
min_ypred = []
max_ypred = []
cond = []
for seed in range(10):
    print(f"Seed: {seed}")
    data_params["seed"] = seed
    res = run(data_params, model_params, attack_params, verbosity_params, other_params, seed)
    acc_l.append(res["accuracy_l"][0])
    min_ypred.append(res["min_ypred"][0])
    max_ypred.append(res["max_ypred"][0])
    cond.append(res["cond"])
print(acc_l)
print(min_ypred)
print(max_ypred)
print(cond)

Seed: 0
Seed: 1
Seed: 2
Seed: 3
Seed: 4
Seed: 5
Seed: 6
Seed: 7
Seed: 8
Seed: 9
[0.5210000276565552, 0.7950000166893005, 0.7790000438690186, 0.7770000100135803, 0.8180000185966492, 0.7910000085830688, 0.7950000166893005, 0.7890000343322754, 0.5380000472068787, 0.8110000491142273]
[-296.59076156107926, -4.52564114908652, -5.673511484411298, -8.380375245901904, -5.813458484371335, -11.09270211128861, -5.931646827522416, -5.443586191531857, -69.78675415576794, -5.146435286173144]
[240.74777676288156, 8.062967090310071, 10.359658009010246, 7.045663003687437, 5.7407786667211695, 5.688827342343328, 5.933379658153002, 5.643244483079846, 57.0348302470963, 5.971274784526761]
[20291603317.12647, 618988173.5184208, 481220517.795961, 379796531.44778395, 407686915.44799507, 849027.1857061002, 853581.9783354799, 992819998.149226, 1871702270.9986954, 817203.3807346252]


In [40]:
# Transductive
data_params["learning_setting"] = "transductive"
acc_l = []
for seed in range(30):
    data_params["seed"] = seed
    res = run(data_params, model_params, attack_params, verbosity_params, other_params, seed)
    acc_l.append(res["accuracy_l"][0])
print(acc_l)
print(np.mean(acc_l))
print(np.std(acc_l))

2023-11-28 11:09:06 (INFO): Starting experiment exp_ntk with configuration:
2023-11-28 11:09:06 (INFO): data_params: {'learning_setting': 'transductive', 'classes': 2, 'n_trn_labeled': 600, 'n_trn_unlabeled': 0, 'n_val': 200, 'n_test': 200, 'sigma': 1, 'avg_within_class_degree': 3.16, 'avg_between_class_degree': 0.74, 'K': 1.5, 'seed': 0}
2023-11-28 11:09:06 (INFO): model_params: {'label': 'GCN', 'model': 'GCN', 'normalization': 'row_normalization', 'depth': 1}
2023-11-28 11:09:06 (INFO): attack_params: {'attack': 'random', 'epsilon_list': [0], 'attack_setting': 'evasion'}
2023-11-28 11:09:06 (INFO): verbosity_params: {'debug_lvl': 'warning'}
2023-11-28 11:09:06 (INFO): other_params: {'device': '0', 'dtype': torch.float64, 'allow_tf32': False}


2023-11-28 11:09:06 (INFO): seed: 0
2023-11-28 11:09:06 (INFO): Currently on gpu device cuda:0
2023-11-28 11:09:06 (INFO): Accuracy 0.6949999928474426 for epsilon 0
2023-11-28 11:09:06 (INFO): Starting experiment exp_ntk with configuration:
2023-11-28 11:09:06 (INFO): data_params: {'learning_setting': 'transductive', 'classes': 2, 'n_trn_labeled': 600, 'n_trn_unlabeled': 0, 'n_val': 200, 'n_test': 200, 'sigma': 1, 'avg_within_class_degree': 3.16, 'avg_between_class_degree': 0.74, 'K': 1.5, 'seed': 1}
2023-11-28 11:09:06 (INFO): model_params: {'label': 'GCN', 'model': 'GCN', 'normalization': 'row_normalization', 'depth': 1}
2023-11-28 11:09:06 (INFO): attack_params: {'attack': 'random', 'epsilon_list': [0], 'attack_setting': 'evasion'}
2023-11-28 11:09:06 (INFO): verbosity_params: {'debug_lvl': 'warning'}
2023-11-28 11:09:06 (INFO): other_params: {'device': '0', 'dtype': torch.float64, 'allow_tf32': False}
2023-11-28 11:09:06 (INFO): seed: 1
2023-11-28 11:09:06 (INFO): Currently on gpu 

[0.6949999928474426, 0.7899999618530273, 0.7599999904632568, 0.8149999976158142, 0.7649999856948853, 0.7849999666213989, 0.7899999618530273, 0.4899999797344208, 0.7899999618530273, 0.5049999952316284, 0.8050000071525574, 0.7199999690055847, 0.7299999594688416, 0.73499995470047, 0.7999999523162842, 0.699999988079071, 0.7749999761581421, 0.7400000095367432, 0.7249999642372131, 0.7699999809265137, 0.8050000071525574, 0.7249999642372131, 0.8050000071525574, 0.7899999618530273, 0.5049999952316284, 0.7400000095367432, 0.6800000071525574, 0.7099999785423279, 0.7549999952316284, 0.7199999690055847]
0.7306666483481725
0.08507578059838189


In [13]:
import numpy as np

In [14]:
a = np.arange(10)
b = np.arange(5)
np.concatenate((a, b))

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4])